In [1]:
import numpy as np
# from gurobipy import *

import gurobipy as gp
from gurobipy import GRB

import time
import pickle
import random
import math
import matplotlib.pyplot as plt
from timeParamCPScores.code.gurobipyTutorial import optimzeTimeAlphasKKT, optimzeTimeAlphasKKTNoMaxLowerBound, optimzeTimeAlphasKKTNoMaxLowerBoundMinArea


PLOT_VALIDATION_TRACES = False
NUM_VALID_TO_PLOT = 100

def computeRValuesIndiv(x_vals,y_vals,x_hats,y_hats):

    R_vals = [ [math.sqrt((x_vals[i][j]-x_hats[i][j])**2 + (y_vals[i][j]-y_hats[i][j])**2) for j in range(len(x_vals[i]))]  for i in range(len(x_vals))  ]

    return R_vals



def computeCPCirlce(x_vals,y_vals,x_hats,y_hats,delta):

    R_vals = [ math.sqrt((x_vals[i]-x_hats[i])**2 + (y_vals[i]-y_hats[i])**2) for i in range(len(x_vals))]

    R_vals.sort()
    R_vals.append(max(R_vals))

    ind_to_ret = math.ceil(len(R_vals)*(1-delta))
    return(R_vals[ind_to_ret])



def computeCPFixedAlphas(x_vals,y_vals,x_hats,y_hats,alphas,delta):

    R_vals = [max([alphas[j]*math.sqrt((x_vals[i][j]-x_hats[i][j])**2 + (y_vals[i][j]-y_hats[i][j])**2) for j in range(len(x_vals[i])) ]) for i in range(len(x_vals))]
    
    R_vals.sort()
    R_vals.append(max(R_vals))

    ind_to_ret = math.ceil(len(R_vals)*(1-delta))
    return R_vals[ind_to_ret]



def computeCoverageRAndAlphas(x_vals,y_vals,x_hats,y_hats,alphas,D_cp):

    R_vals = [max([alphas[j]*math.sqrt((x_vals[i][j]-x_hats[i][j])**2 + (y_vals[i][j]-y_hats[i][j])**2) for j in range(len(x_vals[i])) ]) for i in range(len(x_vals))]

    num_points_within = sum(r <= D_cp for r in R_vals)
    coverage_pct = float(num_points_within)/len(R_vals)
    return coverage_pct


def computeCoverageCircle(x_vals,y_vals,x_hats,y_hats,Ds_cp):
    coverage_count = 0
    for i in range(len(x_vals)):

        temp = sum([1 if math.sqrt((x_vals[i][j]-x_hats[i][j])**2 + (y_vals[i][j]-y_hats[i][j])**2) > Ds_cp[j] else 0 for j in range(len(x_vals[i]))])
        if temp == 0:
            coverage_count += 1
        

    coverage_pct = float(coverage_count)/len(x_vals)
    return coverage_pct


def plot_circle(x, y, size, color="-b", label=None):  # pragma: no cover
    deg = list(range(0, 360, 5))
    deg.append(0)
    xl = [x + size * math.cos(np.deg2rad(d)) for d in deg]
    yl = [y + size * math.sin(np.deg2rad(d)) for d in deg]

    if label is None:
        plt.plot(xl, yl, color)
    else:
        plt.plot(xl, yl, color,label=label)

In [2]:
save_path = "timeParamCPScores/code/data/"

fid = open(save_path + "d_value.pkl", 'rb')
d_value = pickle.load(fid)
fid.close()

fid = open(save_path + "x_value.pkl", 'rb')
x_value = pickle.load(fid)
fid.close()

fid = open(save_path + "y_value.pkl", 'rb')
y_value = pickle.load(fid)
fid.close()

fid = open(save_path + "x_l_value.pkl", 'rb')
x_l_value = pickle.load(fid)
fid.close()

fid = open(save_path + "y_l_value.pkl", 'rb')
y_l_value = pickle.load(fid)
fid.close()



p_len = 20
delta = 0.05



dict_keys_x = list(x_value.keys())
dict_key_x = dict_keys_x[0]
print(dict_key_x)

dict_keys_y = list(y_value.keys())
dict_key_y = dict_keys_y[0]
print(dict_key_y)

dict_keys_x_l = list(x_l_value.keys())
dict_key_x_l = dict_keys_x_l[0]
print(dict_key_x_l)

dict_keys_y_l = list(y_l_value.keys())
dict_key_y_l = dict_keys_y_l[0]
print(dict_key_y_l)


p_len = 20
delta = 0.1
calib_percent = 0.75
trace_for_alphas = 50
makePlots = True

random.seed(34956)

dict_keys_x = list(x_value.keys())
dict_key_x = dict_keys_x[0]
print(dict_key_x)

dict_keys_y = list(y_value.keys())
dict_key_y = dict_keys_y[0]
print(dict_key_y)

dict_keys_x_l = list(x_l_value.keys())
dict_key_x_l = dict_keys_x_l[0]
print(dict_key_x_l)

dict_keys_y_l = list(y_l_value.keys())
dict_key_y_l = dict_keys_y_l[0]
print(dict_key_y_l)


keys_x = list(x_value[dict_key_x].keys())

all_x = []
all_y = []
all_x_l = []
all_y_l = []

all_x_err = []
all_y_err = []


for k in keys_x:

    final_key = list(x_value[dict_key_x][k].keys())[0]

    x = x_value[dict_key_x][k][final_key]
    y = y_value[dict_key_y][k][final_key]
    x_l = x_l_value[dict_key_x_l][k][final_key]
    y_l = y_l_value[dict_key_y_l][k][final_key]

    all_x.append(x[p_len:2*p_len])
    all_y.append(y[p_len:2*p_len])

    all_x_l.append(x_l[0:p_len])
    all_y_l.append(y_l[0:p_len])


## create train/test split
temp = list(zip(all_x, all_y, all_x_l,all_y_l))
random.shuffle(temp)

res1,res2,res3,res4 = zip(*temp)
all_x,all_y,all_x_l,all_y_l = list(res1),list(res2),list(res3),list(res4)

calib_ind = round(len(all_x)*calib_percent)

print("Num calibration data: " + str(calib_ind))
print("Num validation data: " + str(len(all_x)-calib_ind))

all_x_calib = all_x[0:calib_ind]
all_y_calib = all_y[0:calib_ind]
all_x_l_calib = all_x_l[0:calib_ind]
all_y_l_calib = all_y_l[0:calib_ind]


all_x_calib_alphas = all_x_calib[0:trace_for_alphas]
all_y_calib_alphas = all_y_calib[0:trace_for_alphas]
all_x_l_calib_alphas = all_x_l_calib[0:trace_for_alphas]
all_y_l_calib_alphas = all_y_l_calib[0:trace_for_alphas]


all_x_calib_CP = all_x_calib[trace_for_alphas:]
all_y_calib_CP = all_y_calib[trace_for_alphas:]
all_x_l_calib_CP = all_x_l_calib[trace_for_alphas:]
all_y_l_calib_CP = all_y_l_calib[trace_for_alphas:]


all_x_valid = all_x[calib_ind:]
all_y_valid = all_y[calib_ind:]
all_x_l_valid = all_x_l[calib_ind:]
all_y_l_valid = all_y_l[calib_ind:]


print("Num data calib: " + str(len(all_x_calib)))
print("Num data valid: " + str(len(all_x_valid)))


## compute R values of data
R_vals_calib_alpha = computeRValuesIndiv(all_x_calib_alphas,all_y_calib_alphas,all_x_l_calib_alphas,all_y_l_calib_alphas)

## run optimziation
M = 100000 #big value for linearization of max constraint


# optimzeTimeAlphasMinArgminHeuristic(fake_R_vals,delta,M)


start_time = time.time()
m = optimzeTimeAlphasKKTNoMaxLowerBound(R_vals_calib_alpha,delta,M)
# m_min_area = optimzeTimeAlphasKKTNoMaxLowerBoundMinArea(R_vals_calib_alpha,delta,M)
end_time = time.time()


start_time_milp = time.time()
m_milp = optimzeTimeAlphasKKT(R_vals_calib_alpha,delta,M)
end_time_milp = time.time()

# start_time_min_area = time.time()
# m_min_area = optimzeTimeAlphasKKTNoMaxLowerBoundMinArea(R_vals_calib_alpha,delta,M)
# end_time_min_area = time.time()


print("Solve time: " + str(end_time-start_time))
print("Solve time MILP: " + str(end_time_milp-start_time_milp))
# print("Solve time min area: " + str(end_time_min_area-start_time_min_area))

alphas = []    
for v in m.getVars():
    if "alphas" in v.varName:
        alphas.append(v.x)
    if "q" in v.varName:
        # print(v.x)
        print("obj: " + str(v.x))

alphas_milp = []    
for v in m_milp.getVars():
    if "alphas" in v.varName:
        alphas_milp.append(v.x)
    if "q" in v.varName:
        # print(v.x)
        print("obj MILP: " + str(v.x))


print("alphas: " + str(alphas))
print("alphas MILP: " + str(alphas_milp))

# plt.plot(alphas,'k*')
# plt.xlabel("Prediction Horizon",fontsize="16")
# plt.ylabel("Alpha Value",fontsize="16")

# plt.savefig('images/forPaper/alphas.png')

D_cp = computeCPFixedAlphas(all_x_calib_CP,all_y_calib_CP,all_x_l_calib_CP,all_y_l_calib_CP,alphas,delta)
validation_coverage_alphas_method = computeCoverageRAndAlphas(all_x_valid,all_y_valid,all_x_l_valid,all_y_l_valid,alphas,D_cp)

x20
y20
x_LSTM20
y_LSTM20
x20
y20
x_LSTM20
y_LSTM20
Num calibration data: 968
Num validation data: 323
Num data calib: 968
Num data valid: 323
Set parameter Username
Set parameter LicenseID to value 2615263
Academic license - for non-commercial use only - expires 2026-01-26
Set parameter NonConvex to value 2
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: 13th Gen Intel(R) Core(TM) i7-1360P, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 1372 rows, 321 columns and 2632 nonzeros
Model fingerprint: 0x2969881b
Model has 100 quadratic constraints
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-01, 1e+00]
Presolve removed 278 rows and 0 columns

Continuous model is non-convex -- solving as a MIP

Presolve removed 328 rows and 50 columns
Presolve time:

In [3]:
from conformal.all_paths_conformal_pred import all_paths_conformal_pred
from conformal.bucketed_conformal_pred import bucketed_conformal_pred
from conformal.ORCALSTM_nonconfirmity_score_graph import ORCALSTMNonConformityScoreGraph

adj_list = [[i+1] for i in range(p_len)] + [[p_len]]
terminal_vertices = [i for i in range(len(adj_list)) if i in adj_list[i]]

data = dict()
data["x"] = all_x_calib
data["y"] = all_y_calib
data["xh"] = all_x_l_calib
data["yh"] = all_y_l_calib

score_graph = ORCALSTMNonConformityScoreGraph(adj_list, data)

total_buckets = 20
vbs = bucketed_conformal_pred(score_graph, delta, total_buckets, len(all_x_calib))
vb = vbs.buckets[(terminal_vertices[0], total_buckets)]

path_score_quantiles = vb.path_score_quantiles

validation_coverage_bucketed_conf_pred = computeCoverageCircle(all_x_valid,all_y_valid,all_x_l_valid,all_y_l_valid,path_score_quantiles)

min_path, min_path_scores = all_paths_conformal_pred(score_graph, delta, len(all_x_calib))
all_paths_quantiles = [max(min_path_scores) for _ in range(p_len)]

validation_coverage_all_paths_pred = computeCoverageCircle(all_x_valid,all_y_valid,all_x_l_valid,all_y_l_valid,all_paths_quantiles)

In [4]:
print("Validation coverage alphas method: " + str(validation_coverage_alphas_method))
print("Validation coverage bucketed conf pred: " + str(validation_coverage_bucketed_conf_pred))
print("Validation coverage all paths conf pred: " + str(validation_coverage_all_paths_pred))

Validation coverage alphas method: 0.9009287925696594
Validation coverage bucketed conf pred: 0.9907120743034056
Validation coverage all paths conf pred: 0.9040247678018576


In [5]:
print(vb.path_buckets)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2]
